In [1]:
import pathlib
import numpy as np

from aiida import orm, load_profile
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

# BaTiO3
Let's define the BTO structure to use as example for this tutorial.

## Running the `SelfConsistentHubbardWorkChain`

In [2]:
hubbard_data = orm.load_node(2502)  # I load the node from the database instead of always regenerating it
print(hubbard_data.get_quantum_espresso_hubbard_card())

HUBBARD	ortho-atomic
 U	Ti-3d	5.0
 V	Ti-3d	O-2p	1	4	0.0001
 V	Ti-3d	O-2p	1	2	0.0001
 V	Ti-3d	O-2p	1	3	0.0001
 V	Ti-3d	O-2p	1	72	0.0001
 V	Ti-3d	O-2p	1	83	0.0001
 V	Ti-3d	O-2p	1	114	0.0001
 V	Ti-3d	O-2p	1	69	0.0001



In [3]:
# pw_code = orm.load_code('qe-dev-pw@lumi-small')
# hp_code = orm.load_code('qe-dev-hp@lumi-small')
import os
pw_code = orm.load_code('qe-7.2-pw@theospc24')
hp_code = orm.load_code('qe-7.2-hp@theospc24')

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='fast',
    overrides=pathlib.Path(os.path.join('..', 'yaml_files', 'serial_overrides.yaml'))
)

builder.skip_first_relax = True

In [4]:
new_schema_submit = submit(builder)
print(new_schema_submit)

uuid: 06f5a8b6-8ec3-41f1-a1f8-412f6b690e9c (pk: 9441) (aiida.workflows:quantumespresso.hp.hubbard)


In [5]:
# qe72_submit = 9240
# qe71_submit = 9277